In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import numpy as np
import os

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

from torchvision.utils import make_grid
from torch.utils.data import Dataset, DataLoader, random_split
from torchvision import datasets, transforms, models

from dataset import ContrastiveDataset

import matplotlib.pyplot as plt
%matplotlib inline

from sklearn.manifold import TSNE

In [3]:
seed = 42 
np.random.seed(seed)
torch.manual_seed(seed)

In [4]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

## Dataloader for self-supervised case

In [15]:
transform = transforms.Compose(
    [transforms.ToTensor(), transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))]
)

cifar_train = datasets.CIFAR10(
    root="./data", train=True, download=True, transform=transform
)
cifar_test = datasets.CIFAR10(
    root="./data", train=False, download=True, transform=transform
)

train_img_array = np.array([np.array(image) for image, _ in cifar_train])
test_img_array = np.array([np.array(image) for image, _ in cifar_test])

train_dataset = ContrastiveDataset("train", train_img_array[:40000])
val_dataset = ContrastiveDataset("val", train_img_array[40000:])
test_dataset = ContrastiveDataset("test", train_img_array)

Files already downloaded and verified
Files already downloaded and verified


In [ ]:
ssl_batch_size = 128
train_dataloader = DataLoader(
    train_dataset, batch_size=ssl_batch_size, shuffle=True, num_workers=2
)
val_dataloader = DataLoader(
    val_dataset, batch_size=ssl_batch_size, shuffle=False, num_workers=2
)
test_dataloader = DataLoader(
    test_dataset, batch_size=ssl_batch_size, shuffle=False, num_workers=2
)

In [ ]:
class ProjectionHead(nn.Module):
    def __init__(
        self, in_features: int, hidden_features: int, out_features: int, **kwargs
    ):
        super(ProjectionHead, self).__init__(**kwargs)
        self.linear1 = nn.Linear(
            in_features=in_features, out_features=hidden_features, bias=False
        )
        self.linear2 = nn.Linear(
            in_features=hidden_features, out_features=out_features, bias=False
        )

        self.relu = nn.ReLU()
        self.batch_normalization1 = nn.BatchNorm1d(num_features=hidden_features)
        self.batch_normalization2 = nn.BatchNorm1d(num_features=out_features)

    def forward(self, x):
        x = self.linear1(x)
        x = self.batch_normalization1(x)

        x = self.relu(x)
        x = self.linear2(x)
        x = self.batch_normalization2(x)
        return x

In [ ]:
class BackboneModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.resnet = models.resnet50()

        # Changing the layers like mentioned in the article
        self.resnet.maxpool = nn.Sequential()
        self.resnet.fc = nn.Sequential()
        self.resnet.conv1 = nn.Conv2d(
            3, 64, kernel_size=(3, 3), stride=(1, 1), bias=False
        )

        self.projection_head = ProjectionHead(2048, 2048, 128)
    
    def forward(self, x): 
        x = self.resnet(x)
        x = self.projection_head(torch.squeeze(x))
        return x